In [1]:
# import dependency
from promptflow.azure import PFClient
import json
import os

from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

In [2]:
# set AML workspace 
# east us workspace

sub = "74eccef0-4b8d-4f83-b5f9-fa100d155b22"
rg = "lisal-dev"
ws = "lisal-amlservice"

runtime = "lisal-pf-new-ci"
connection_mapping = {"code_gen": {"connection": "my_azure_open_ai_connection", "deployment_name": "gpt-35-turbo"}}


# canary workspace
"""
sub = "96aede12-2f73-41cb-b983-6d11a904839b" 
rg = "promptflow" 
ws = "promptflow-canary-dev" 

connection_mapping = {"code_gen": {"connection": "azure_open_ai_connection", "deployment_name": "gpt-35-turbo"}}
"""

'\nsub = "96aede12-2f73-41cb-b983-6d11a904839b" \nrg = "promptflow" \nws = "promptflow-canary-dev" \n\nconnection_mapping = {"code_gen": {"connection": "azure_open_ai_connection", "deployment_name": "gpt-35-turbo"}}\n'

In [3]:
# init ml_client

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Get a handle to workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=sub,  # this will look like xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
    resource_group_name=rg,
    workspace_name=ws,
)

# init pf client
pf = PFClient(ml_client)

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot.this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: Azure Active Directory error '(invalid_grant) AADSTS700082: The refresh token has expired due to inactivity. The token was issued on 2023-05-18T04:38:57.5927129Z and was inactive for 90.00:00:00.
Trace ID: d498d020-7e62-4846-8792-dac4a2434300
Correlation ID: c0938e37-1c5d-4f29-ab38-56cda006dd78
Timestamp: 2023-08-23 05:32:45Z'
Content: {"error":"invalid_grant","error_description":"AADSTS700082: The refresh token has expired due to inactivity. The token was issued on 2023-05-18T04:38:57.5927129Z and was inactive for 

In [4]:
# batch run of maths to code
file_path = os.getcwd() #os.path.dirname(os.path.abspath(__file__))

# batch run of maths-to-code
flow = "/".join([file_path, "../../maths-to-code"])
data = "/".join([file_path, "../../maths-to-code/test_data/math_data.jsonl"])

print("\n\n===   Running batch run of maths-to-code   ===\n")
base_run = pf.run(
    flow = flow,
    data = data,
    column_mapping = {"math_question": "${data.question}"},
    connections = connection_mapping,
    runtime = runtime,
    stream = True,
)




===   Running batch run of maths-to-code   ===



Portal url: https://ml.azure.com/prompts/flow/bulkrun/run/maths_to_code_default_20230823_133313_705684/details?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourceGroups/lisal-dev/providers/Microsoft.MachineLearningServices/workspaces/lisal-amlservice&flight=promptfilestorage,PFSourceRun=false
2023-08-23 05:33:30 +0000      77 promptflow-runtime INFO     [maths_to_code_default_20230823_133313_705684] Receiving v2 bulk run request 128d57c2-ba48-4d6a-912b-7a564effbcdf: {"flow_id": "maths_to_code_default_20230823_133313_705684", "flow_run_id": "maths_to_code_default_20230823_133313_705684", "flow_source": {"flow_source_type": 1, "flow_source_info": {"snapshot_id": "6cfaa91d-2a57-412e-bb85-1202f44158e5"}, "flow_dag_file": "flow.dag.yaml"}, "connections": "**data_scrubbed**", "log_path": "https://lisalamlservic0529017975.blob.core.windows.net/azureml/ExperimentRun/dcid.maths_to_code_default_20230823_133313_705684/logs/azureml/executionlogs.txt?sv=2019-07-07&sr=b&sig=**data_scru

In [5]:
# invoke an evaluation run against base run
eval_flow = "/".join([file_path, "../../maths-to-code_accuracy_eval"])

print("\n\n###   Evaluating against the batch run   ###\n")
eval_run = pf.run(
    flow = eval_flow, 
    data = data, 
    run = base_run,
    column_mapping={"groundtruth": "${data.answer}", "prediction": "${run.outputs.answer}"},
    runtime = runtime,
    stream = True,
)

metrics = pf.get_metrics(eval_run)
print(json.dumps(metrics, indent=4))



###   Evaluating against the batch run   ###



Portal url: https://ml.azure.com/prompts/flow/bulkrun/run/maths_to_code_accuracy_eval_default_20230823_133615_271156/details?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourceGroups/lisal-dev/providers/Microsoft.MachineLearningServices/workspaces/lisal-amlservice&flight=promptfilestorage,PFSourceRun=false
2023-08-23 05:36:28 +0000      77 promptflow-runtime INFO     [maths_to_code_accuracy_eval_default_20230823_133615_271156] Receiving v2 bulk run request 73e42a3e-0417-4a9e-ba6d-c968d95b9cc8: {"flow_id": "maths_to_code_accuracy_eval_default_20230823_133615_271156", "flow_run_id": "maths_to_code_accuracy_eval_default_20230823_133615_271156", "flow_source": {"flow_source_type": 1, "flow_source_info": {"snapshot_id": "bff0a386-6afd-4ed8-9038-a11a7607bfd9"}, "flow_dag_file": "flow.dag.yaml"}, "log_path": "https://lisalamlservic0529017975.blob.core.windows.net/azureml/ExperimentRun/dcid.maths_to_code_accuracy_eval_default_20230823_133615_271156/logs/azureml/executionlogs.txt?